In [1]:
import numpy as np
from QAOA import QAOA_class as QA
import networkx as nx
import matplotlib.pyplot as plt 
from IPython.display import clear_output

# Function to create a random QUBO instance:

## We consider a general QUBO function $f(\boldsymbol{x}) = \sum_{i<j} J_{i,j} x_i x_j + \sum_{i} h_{i} x_i$, where $J_{i,j}, h_i \in [-10,10]$ and $x_i \in [-1,1]$. The code below considers an Erdos-Renyi random graph with random edges on 'n' nodes to fix the couplings $\langle i,j \rangle$. Thereafter we sample, $J_{i,j}, h_i$ from a unifrom distribution over $[-10,10]$ to construct a random instance. Next, we pass the couplings to a function which generates the diagonal of a Hamiltonian. The Hamiltonian is straigtforwardly constructed by following a vector space embedding: 
\begin{equation}
x_i \to Z_i, \\
\times \to \otimes.
\end{equation}

In [7]:
def QUBO_inst_rand(n): 
    m = np.random.randint(1, high = n*(n-1)/2)
    G = nx.gnm_random_graph(n, m)
    edges = list(G.edges)
    edge_info = [(*x, np.random.randint(low = -10, high = 10)) for x in edges]
    diags = [np.random.randint(low = -10, high = 10) for x in np.arange(n)]
    return edge_info, diags

def QUBO_Hamiltonian(n, edge_info, diags):
    Z, Id, H = [1,-1], [1,1], np.zeros(2**n) 
    
    def tensor(k):
        t = k[0]
        i = 1
        while i < len(k) :
            t = np.kron(t,k[i])
            i+=1
        return t
    
    for x in edge_info:
        tensor_array = [Id]*n 
        tensor_array[x[0]] = Z
        tensor_array[x[1]] = Z
        H+= x[2]*tensor(tensor_array)
    
    for i in range(n):
        tensor_array = [Id]*n 
        tensor_array[i] = Z
        H+= diags[i]*tensor(tensor_array)
        
    return H

In [225]:
n = 8
Adj, Diag = QUBO_inst_rand(n)
H = QUBO_Hamiltonian(n, Adj, Diag)
E_min = np.min(H)
optimal_idx = np.where(H == E_min)[0]
optimal_assignments = [np.binary_repr(x , n) for x in optimal_idx]
print(f'optimal_assignments: {optimal_assignments}\nmin_QUBO: {E_min}\n\n')
Depth = 5
Q = QA.QAOA(Depth, H)
Q.run_heuristic_LW()
print(' QAOA:',end = '\n')
print(Q.log)

optimal_assignments: ['00110001']
min_QUBO: -51.0


 QAOA:
 Depth: 5 
 Error: 0.8121005596433974 
 QAOA_Eg: -50.1878994403566 
 Exact_Eg: -51.0 
 Overlap: 0.8418207922877954 
 Exe_time: 1.2683489322662354 
 Iternations: 583.0
